# Skolregistret Stockholms stadsarkiv
**Status**: started test webcrawl, version 0.51

**Task:** del i [#1 Svenska skolor i Wikidata och i Bildhistoria.se](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/issues/1)
att kunna hämta ut skolor från Wikidata... status idag är att Wikidata har bra information om Skolenheter 
* se [Wikidata:WikiProject_Sweden/Schools](https://www.wikidata.org/wiki/Wikidata:WikiProject_Sweden/Schools) 
* äldre skolor/skolformer är det sämre se [Wikidata:WikiProject_Sweden/Svenska_skolor](https://www.wikidata.org/wiki/Wikidata:WikiProject_Sweden/Svenska_skolor)
  
 
* [Denna Notebook](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/blob/main/Notebook/Skolregistret%20Stockholms%20stadsarkiv.ipynb)
 * [skapad csv fil](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/blob/main/Notebook/SkolregistretStockholmsStadsarkiv.csv) 
   * [small csv](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/blob/main/Notebook/SkolregistretStockholmsStadsarkiv_small.csv) 
   * [extra allt csv](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/blob/main/Notebook/SkolregistretStockholmsStadsarkiv_allt.csv) 
   
Databasen "Skolregistret Stockholms stadsarkiv" copyright okänd, har besökt stadsarkivet och ombads skicka fråga med email, väntar svar
* [sok.stadsarkivet.stockholm.se/Databas/skolregistret](https://sok.stadsarkivet.stockholm.se/Databas/skolregistret/Sok) 1354 poster

**Version history**
* 0.2 fixed bug with matched classes
* 0.3 extrahera SSA arkiv nummer och hämtade fler fält
* 0.4 hämtar från Riksarkivet agentid och namn
* 0.5 se om vi kan testa Riksarkivets API skickade även en fråga till dom hur vi får ut Sveriges alla skolor över tid från Arkiven se [Riksarkivet/dataplattform/issues/1](https://github.com/Riksarkivet/dataplattform/issues/1) se även fråga hur vi gör detta i WD [Talk:Q89788826](https://www.wikidata.org/wiki/Talk:Q89788826)

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-05-14 16:22:15.030620


In [2]:
import urllib3, re, time
import pandas as pd 
from bs4 import BeautifulSoup
from csv import DictReader, DictWriter
from tqdm import tqdm
req = urllib3.PoolManager()
    
urlbase = "https://sok.stadsarkivet.stockholm.se/Databas/skolregistret/Sok?sidindex="
skolKolumner = ["Url","Namn","Alt. namn","Byggår och arkitekt","Alternativt namn","Annexskolor",
           "Adress","Arkiv på Stadsarkivet","Rektorsområde","Arkiv hos övriga","Historik"]
dfTot = pd.DataFrame(columns=skolKolumner)
for i in tqdm(range(0,1300)):
    try:
        url = urlbase + str(i)
        res = req.request('GET', url)
        soup = BeautifulSoup(res.data, 'html.parser')
        classes = ["arende-table-row arende-table-row-alternating", 
                   "arende-table-row arende-table-row-nonalternating"]
        contents = soup.find_all(class_ = classes)
        i = 0 
        for tablerow in contents:
            for rowLink in tablerow.find_all('a', href=True):
                schoolUrl = rowLink['href'].split("?sidindex")[0]
                namnUrl = rowLink.string
                
             
            td_list = tablerow.find_all("td")
            namn = td_list[0].text.strip()
            altnamn = td_list[1].text.strip()
            adress = td_list[2].text.strip()
            historik = td_list[3].text.strip()
            #print(namn,altnamn,adress,historik)
                
            dfTot = dfTot.append({'Url': schoolUrl, 
                                  'Namn': namn, 
                                  'Alt. namn': altnamn,
                                  'Historik':historik,
                                  'Adress':adress
                                  }, ignore_index=True)
    except Exception as e: 
        print(e)


100%|██████████| 1300/1300 [06:33<00:00,  3.30it/s]


In [3]:
dfTot.shape

(1354, 11)

In [4]:
dfTot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1354 entries, 0 to 1353
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Url                    1354 non-null   object
 1   Namn                   1354 non-null   object
 2   Alt. namn              1354 non-null   object
 3   Byggår och arkitekt    0 non-null      object
 4   Alternativt namn       0 non-null      object
 5   Annexskolor            0 non-null      object
 6   Adress                 1354 non-null   object
 7   Arkiv på Stadsarkivet  0 non-null      object
 8   Rektorsområde          0 non-null      object
 9   Arkiv hos övriga       0 non-null      object
 10  Historik               1354 non-null   object
dtypes: object(11)
memory usage: 116.5+ KB


In [5]:
dfTot

,Url,Namn,Alt. namn,Byggår och arkitekt,Alternativt namn,Annexskolor,Adress,Arkiv på Stadsarkivet,Rektorsområde,Arkiv hos övriga,Historik
0,Visa/carlgrenska-waldorfgymnasiet/bf4aeb94-88b...,Carlgrenska Waldorfgymnasiet,,NaN,NaN,NaN,Skebokvarnsvägen 370\r\ntidigare Tegeluddsväge...,NaN,NaN,NaN,Fristående gymnasieskola\r\n\r\nÖppnad ht 2013...
1,Visa/larsboda-ro--1960-1991/7bb2e5c7-e0aa-4754...,Larsboda ro 1960-1991,,NaN,NaN,NaN,,NaN,NaN,NaN,Huvudskola:\r\nLarsbodaskolan 1960-1991
2,Visa/elektrum/5240aebb-c129-4615-9474-006c5b4e...,Elektrum,Electrum,NaN,NaN,NaN,Kistagången 14-18,NaN,NaN,NaN,Gymnasium\r\n\r\nAnnex till Vasa gymnasium 198...
3,Visa/slattgardsskolan/51e42e66-35c7-4a49-9355-...,Slättgårdsskolan,,NaN,NaN,NaN,Frimurarvägen 11-17,NaN,NaN,NaN,Grundskola åk 1-9 (före ht 2005 1-6)\r\n\r\nEg...
4,Visa/lovsatra-resursskola/de119201-dadb-4815-a...,Lövsätra resursskola,Skärholmens skoldaghem / Skärholmens resursskola,NaN,NaN,NaN,1971-2002 Falkholmsgränd 41 (tidigare Stånghol...,NaN,NaN,NaN,Specialskola: Grundskola åk 1-6\r\n\r\nÖppnad ...
...,...,...,...,...,...,...,...,...,...,...,...
1349,Visa/kunskapsskolan-tyreso/e2dbbd7c-a06a-47ce-...,Kunskapsskolan Tyresö,,NaN,NaN,NaN,Solkraftsvägen 15,NaN,NaN,NaN,Fristående grundskola åk 6-9.\r\n\r\nSkolan st...
1350,Visa/sodermalms-privata-skola/0f0c2328-a79d-46...,Södermalms privata skola,,NaN,NaN,NaN,1938 anges Folkungagatan 55,NaN,NaN,NaN,Privatskola
1351,Visa/olof-fryxells-flickskola/7c1a21e8-8f27-45...,Olof Fryxells flickskola,Bataljonspredikanten Olof Fryxells flickskola,NaN,NaN,NaN,Järntorget 80,NaN,NaN,NaN,Anges finnas 1848 på given adress i Staden mel...
1352,Visa/martinskolan/fcb2590d-bda7-4c70-9616-fee8...,Martinskolan,Martinskolan Söders waldorfskola 7 / (Martinsk...,NaN,NaN,NaN,Munstycksvägen 18,NaN,NaN,NaN,"Fristående Waldorfskola åk F-9.\r\nHuvudman, S..."


In [6]:
dfTot.to_csv("SkolregistretStockholmsStadsarkiv.csv")

In [7]:
#dfTot[{"Namn","Url","Alt. namn","Adress"}].to_csv("SkolregistretStockholmsStadsarkiv_small.csv")

In [8]:
# IS A BUG preserving orders -->  https://stackoverflow.com/questions/15653688/preserving-column-order-in-python-pandas-dataframe
dfTot.to_csv("SkolregistretStockholmsStadsarkiv_small.csv",
             header=True,columns=["Namn","Url","Alt. namn","Adress"],
            sep=",")

In [9]:
#Get more data 
urlBaseView ="https://sok.stadsarkivet.stockholm.se/Databas/skolregistret/"
Viewids =["MainContent_alternativt-namn" ,"MainContent_namn",
        "MainContent_adress","MainContent_byggar-och-arkitekt","MainContent_annexskolor",
        "MainContent_arkiv-pa-stadsarkivet","MainContent_arkiv-hos-ovriga",
        "MainContent_historik","MainContent_rektorsomrade"]

# Field
contentList=[
{"field":"Namn",
"content":"MainContent_Namn_visa_textbox"},
{"field":"Alt. namn",
"content":"MainContent_Alternativt namn_visa_textbox"},
{"field":"Adress",
"content":"MainContent_Adress_visa_textbox"},
{"field":"Rektorsområde",
"content":"MainContent_Rektorsområde_visa_textbox"},
{"field":"Byggår och arkitekt",
"content":"MainContent_Byggår och arkitekt_visa_textbox"},
{"field":"Annexskolor",
"content":"MainContent_Annexskolor_visa_textbox"},
{"field":"Arkiv på Stadsarkivet",
"content":"MainContent_Arkiv på Stadsarkivet_visa_textbox"},
{"field":"Arkiv hos övriga",
"content":"MainContent_Arkiv hos övriga_visa_textbox"},
{"field":"Historik",
"content":"MainContent_Historik_visa_textbox"}]

for index, row in tqdm(dfTot.iterrows(), total=dfTot.shape[0]):
    #print(index,row["Url"])
    itemUrl = urlBaseView + row["Url"]
    #print(itemUrl)
    res = req.request('GET', itemUrl)
    soupSchool = BeautifulSoup(res.data, 'html.parser')
    for c in contentList:
        tag = soupSchool.find(id=c["content"])
        #print("\t\t",c["field"],len(tag.text.strip()),"\t",tag.text.strip())
        if len(tag.text.strip()) > 0:
            dfTot.loc[index,c["field"]] = tag.text.strip()

    

100%|██████████| 1354/1354 [06:41<00:00,  3.38it/s]


In [10]:
dfTot.shape

(1354, 11)

In [11]:
dfTot

,Url,Namn,Alt. namn,Byggår och arkitekt,Alternativt namn,Annexskolor,Adress,Arkiv på Stadsarkivet,Rektorsområde,Arkiv hos övriga,Historik
0,Visa/carlgrenska-waldorfgymnasiet/bf4aeb94-88b...,Carlgrenska Waldorfgymnasiet,,NaN,NaN,NaN,Skebokvarnsvägen 370\r\ntidigare Tegeluddsväge...,SSA/7131: e-arkiv: slutbetyg 2016-2019. Vissa ...,NaN,NaN,Fristående gymnasieskola\r\n\r\nÖppnad ht 2013...
1,Visa/larsboda-ro--1960-1991/7bb2e5c7-e0aa-4754...,Larsboda ro 1960-1991,,NaN,NaN,"Hästhagsskolan 1975-1991, \r\nMagelungsskolan ...",,SSA/1406: Larsboda ro 1960-1992.,NaN,NaN,Huvudskola:\r\nLarsbodaskolan 1960-1991
2,Visa/elektrum/5240aebb-c129-4615-9474-006c5b4e...,Elektrum,Electrum,NaN,NaN,NaN,Kistagången 14-18,SSA/1331: Vasa gymnasium 1987-1990.,NaN,Thorildsplans gymnasium 1990-1992 ?,Gymnasium\r\n\r\nAnnex till Vasa gymnasium 198...
3,Visa/slattgardsskolan/51e42e66-35c7-4a49-9355-...,Slättgårdsskolan,,1968 Uddén-Wåhlström,NaN,NaN,Frimurarvägen 11-17,SSA/2462: Mälarhöjdens ro 1966-1989.,"Mälarhöjdens ro 1966-1989, \r\nBredängs ro 198...",Bredängsskolan (ro) 1989-1991. \r\nSlättgårdss...,Grundskola åk 1-9 (före ht 2005 1-6)\r\n\r\nEg...
4,Visa/lovsatra-resursskola/de119201-dadb-4815-a...,Lövsätra resursskola,Skärholmens skoldaghem / Skärholmens resursskola,NaN,NaN,NaN,1971-2002 Falkholmsgränd 41 (tidigare Stånghol...,"SSA/2060: S:t Örjans skolor 1971-1985, 1991-2009.","S:t Örjans ro 1971-1985, \r\nÖsterholms ro 198...",Österholmsskolan (ro) 1985-1990,Specialskola: Grundskola åk 1-6\r\n\r\nÖppnad ...
...,...,...,...,...,...,...,...,...,...,...,...
1349,Visa/kunskapsskolan-tyreso/e2dbbd7c-a06a-47ce-...,Kunskapsskolan Tyresö,,NaN,NaN,NaN,Solkraftsvägen 15,SSA/7093: e-arkiv: slutbetyg 2012-2021.,NaN,I övrigt på skolan.,Fristående grundskola åk 6-9.\r\n\r\nSkolan st...
1350,Visa/sodermalms-privata-skola/0f0c2328-a79d-46...,Södermalms privata skola,,NaN,NaN,NaN,1938 anges Folkungagatan 55,Folkskoledirektionen (SSA/0569) - E III a3 : d...,NaN,NaN,Privatskola
1351,Visa/olof-fryxells-flickskola/7c1a21e8-8f27-45...,Olof Fryxells flickskola,Bataljonspredikanten Olof Fryxells flickskola,NaN,NaN,NaN,Järntorget 80,NaN,NaN,okänt,Anges finnas 1848 på given adress i Staden mel...
1352,Visa/martinskolan/fcb2590d-bda7-4c70-9616-fee8...,Martinskolan,Martinskolan Söders waldorfskola 7 / (Martinsk...,NaN,NaN,NaN,Munstycksvägen 18,SSA/7059: e-arkiv: slutbetyg 2012-2021. Lucka ...,NaN,I övrigt på skolan.,"Fristående Waldorfskola åk F-9.\r\nHuvudman, S..."


In [12]:
dfTot["Arkiv på Stadsarkivet"] 


0       SSA/7131: e-arkiv: slutbetyg 2016-2019. Vissa ...
1                        SSA/1406: Larsboda ro 1960-1992.
2                     SSA/1331: Vasa gymnasium 1987-1990.
3                    SSA/2462: Mälarhöjdens ro 1966-1989.
4       SSA/2060: S:t Örjans skolor 1971-1985, 1991-2009.
                              ...                        
1349              SSA/7093: e-arkiv: slutbetyg 2012-2021.
1350    Folkskoledirektionen (SSA/0569) - E III a3 : d...
1351                                                  NaN
1352    SSA/7059: e-arkiv: slutbetyg 2012-2021. Lucka ...
1353                SSA/2374: Ekensbergsskolan 1991-2013.
Name: Arkiv på Stadsarkivet, Length: 1354, dtype: object

In [13]:
# Find columns with an Archive id SSA/
dfTot[dfTot["Arkiv på Stadsarkivet"].str.contains("SSA/",na=False)]

,Url,Namn,Alt. namn,Byggår och arkitekt,Alternativt namn,Annexskolor,Adress,Arkiv på Stadsarkivet,Rektorsområde,Arkiv hos övriga,Historik
0,Visa/carlgrenska-waldorfgymnasiet/bf4aeb94-88b...,Carlgrenska Waldorfgymnasiet,,NaN,NaN,NaN,Skebokvarnsvägen 370\r\ntidigare Tegeluddsväge...,SSA/7131: e-arkiv: slutbetyg 2016-2019. Vissa ...,NaN,NaN,Fristående gymnasieskola\r\n\r\nÖppnad ht 2013...
1,Visa/larsboda-ro--1960-1991/7bb2e5c7-e0aa-4754...,Larsboda ro 1960-1991,,NaN,NaN,"Hästhagsskolan 1975-1991, \r\nMagelungsskolan ...",,SSA/1406: Larsboda ro 1960-1992.,NaN,NaN,Huvudskola:\r\nLarsbodaskolan 1960-1991
2,Visa/elektrum/5240aebb-c129-4615-9474-006c5b4e...,Elektrum,Electrum,NaN,NaN,NaN,Kistagången 14-18,SSA/1331: Vasa gymnasium 1987-1990.,NaN,Thorildsplans gymnasium 1990-1992 ?,Gymnasium\r\n\r\nAnnex till Vasa gymnasium 198...
3,Visa/slattgardsskolan/51e42e66-35c7-4a49-9355-...,Slättgårdsskolan,,1968 Uddén-Wåhlström,NaN,NaN,Frimurarvägen 11-17,SSA/2462: Mälarhöjdens ro 1966-1989.,"Mälarhöjdens ro 1966-1989, \r\nBredängs ro 198...",Bredängsskolan (ro) 1989-1991. \r\nSlättgårdss...,Grundskola åk 1-9 (före ht 2005 1-6)\r\n\r\nEg...
4,Visa/lovsatra-resursskola/de119201-dadb-4815-a...,Lövsätra resursskola,Skärholmens skoldaghem / Skärholmens resursskola,NaN,NaN,NaN,1971-2002 Falkholmsgränd 41 (tidigare Stånghol...,"SSA/2060: S:t Örjans skolor 1971-1985, 1991-2009.","S:t Örjans ro 1971-1985, \r\nÖsterholms ro 198...",Österholmsskolan (ro) 1985-1990,Specialskola: Grundskola åk 1-6\r\n\r\nÖppnad ...
...,...,...,...,...,...,...,...,...,...,...,...
1348,Visa/kungsholmens-skola/67ff287c-0dae-484d-be2...,Kungsholmens skola,Kungsholmens folkskola; Kungsholms folkskola,se Historik mm !,NaN,NaN,Mariebergsgatan 34,SSA/0232A: Kungsholms folkskolor/ro 1847-1973....,"Kungsholms församling -1903, \r\nKungsholms öd...",NaN,"Folkskola, Kommunal mellanskola, Enhetsskola, ..."
1349,Visa/kunskapsskolan-tyreso/e2dbbd7c-a06a-47ce-...,Kunskapsskolan Tyresö,,NaN,NaN,NaN,Solkraftsvägen 15,SSA/7093: e-arkiv: slutbetyg 2012-2021.,NaN,I övrigt på skolan.,Fristående grundskola åk 6-9.\r\n\r\nSkolan st...
1350,Visa/sodermalms-privata-skola/0f0c2328-a79d-46...,Södermalms privata skola,,NaN,NaN,NaN,1938 anges Folkungagatan 55,Folkskoledirektionen (SSA/0569) - E III a3 : d...,NaN,NaN,Privatskola
1352,Visa/martinskolan/fcb2590d-bda7-4c70-9616-fee8...,Martinskolan,Martinskolan Söders waldorfskola 7 / (Martinsk...,NaN,NaN,NaN,Munstycksvägen 18,SSA/7059: e-arkiv: slutbetyg 2012-2021. Lucka ...,NaN,I övrigt på skolan.,"Fristående Waldorfskola åk F-9.\r\nHuvudman, S..."


In [14]:
dfTot.to_csv("SkolregistretStockholmsStadsarkiv_allt.csv")

In [15]:
dfTot["SSA"]="SSA/" + dfTot["Arkiv på Stadsarkivet"].str.extract('SSA/(\d*)')


### SSA numret används hos Riksarkivet
SSA numret används hos Riksarkivet --> WD egenskap [Property:P5324](https://www.wikidata.org/wiki/Property:P5324) gissar att Riksarkivet inte har ett API men borde kunna gå att hämta [Property:P9713](https://www.wikidata.org/wiki/Property:P9713)

Exempel 
* SSA/7131 --> [SE/SSA/7131](https://sok.riksarkivet.se/?postid=ArkisRef%20SE/SSA/7131) --> agent https://sok.riksarkivet.se/arkiv/OSjzkpBeXQPSC4OvnXVJg6


In [16]:
dfTot["SSA"]

0       SSA/7131
1       SSA/1406
2       SSA/1331
3       SSA/2462
4       SSA/2060
          ...   
1349    SSA/7093
1350    SSA/0569
1351         NaN
1352    SSA/7059
1353    SSA/2374
Name: SSA, Length: 1354, dtype: object

In [17]:
dfTot.to_csv("SkolregistretStockholmsStadsarkiv_allt2.csv")

In [18]:
dfTot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1354 entries, 0 to 1353
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Url                    1354 non-null   object
 1   Namn                   1354 non-null   object
 2   Alt. namn              1354 non-null   object
 3   Byggår och arkitekt    325 non-null    object
 4   Alternativt namn       0 non-null      object
 5   Annexskolor            204 non-null    object
 6   Adress                 1354 non-null   object
 7   Arkiv på Stadsarkivet  947 non-null    object
 8   Rektorsområde          262 non-null    object
 9   Arkiv hos övriga       731 non-null    object
 10  Historik               1354 non-null   object
 11  SSA                    898 non-null    object
dtypes: object(12)
memory usage: 127.1+ KB


### Test hämta Riksarkivets agentID och arkiv namn

In [19]:
baseUrlRiksarkiv = "https://sok.riksarkivet.se/?postid=ArkisRef%20"

def isnotNaN(value):
    return value == value

for index, row in tqdm(dfTot.iterrows(), total=dfTot.shape[0]):
    if isnotNaN(row["SSA"]):
        urlRA = baseUrlRiksarkiv + "SE/" + row["SSA"]
     
        try: 
            resRA = req.request('GET', urlRA)
            #print("\tStatus:", resRA.status)
            #Check if more than 3000 character Riksarkivet skickar alltid status 200...
            if len(resRA.data) > 3000:
                soupRA = BeautifulSoup(resRA.data, 'html.parser')
                title = soupRA.find_all("h1")
                dfTot.loc[index,"RAarkivnamn"] = title

                copyBtn = soupRA.find("i",class_="copy-btn")
                if copyBtn['data-copy-value'] is not None:
                    agentid = copyBtn['data-copy-value'].replace("https://sok.riksarkivet.se/arkiv/","")
                    dfTot.loc[index,"agentid"] = agentid
                else:
                    print("\tError agent is None: ",urlRA)
                #print("\tAgentid\t",agentid)
 
        except Exception as e: 
            print( e)
            print(urlRA)


100%|██████████| 1354/1354 [08:25<00:00,  2.68it/s]


In [28]:
dfTot

,Url,Namn,Alt. namn,Byggår och arkitekt,Alternativt namn,Annexskolor,Adress,Arkiv på Stadsarkivet,Rektorsområde,Arkiv hos övriga,Historik,SSA,RAarkivnamn,agentid
0,Visa/carlgrenska-waldorfgymnasiet/bf4aeb94-88b...,Carlgrenska Waldorfgymnasiet,,NaN,NaN,NaN,Skebokvarnsvägen 370\r\ntidigare Tegeluddsväge...,SSA/7131: e-arkiv: slutbetyg 2016-2019. Vissa ...,NaN,NaN,Fristående gymnasieskola\r\n\r\nÖppnad ht 2013...,SSA/7131,[\r\nCarlgrenska Waldorfgymnasiet\r\n ],OSjzkpBeXQPSC4OvnXVJg6
1,Visa/larsboda-ro--1960-1991/7bb2e5c7-e0aa-4754...,Larsboda ro 1960-1991,,NaN,NaN,"Hästhagsskolan 1975-1991, \r\nMagelungsskolan ...",,SSA/1406: Larsboda ro 1960-1992.,NaN,NaN,Huvudskola:\r\nLarsbodaskolan 1960-1991,SSA/1406,[\r\nLarsboda rektorsområde\r\n ],NSaspJigr4AywCL8qpLfp6
2,Visa/elektrum/5240aebb-c129-4615-9474-006c5b4e...,Elektrum,Electrum,NaN,NaN,NaN,Kistagången 14-18,SSA/1331: Vasa gymnasium 1987-1990.,NaN,Thorildsplans gymnasium 1990-1992 ?,Gymnasium\r\n\r\nAnnex till Vasa gymnasium 198...,SSA/1331,[\r\nVasa gymnasium\r\n ],VZ6k9IgabKQ8ySqb1EfBm3
3,Visa/slattgardsskolan/51e42e66-35c7-4a49-9355-...,Slättgårdsskolan,,1968 Uddén-Wåhlström,NaN,NaN,Frimurarvägen 11-17,SSA/2462: Mälarhöjdens ro 1966-1989.,"Mälarhöjdens ro 1966-1989, \r\nBredängs ro 198...",Bredängsskolan (ro) 1989-1991. \r\nSlättgårdss...,Grundskola åk 1-9 (före ht 2005 1-6)\r\n\r\nEg...,SSA/2462,[\r\nMälarhöjdens rektorsområde\r\n ],a82JBOhltasB7JdRHUzTQ4
4,Visa/lovsatra-resursskola/de119201-dadb-4815-a...,Lövsätra resursskola,Skärholmens skoldaghem / Skärholmens resursskola,NaN,NaN,NaN,1971-2002 Falkholmsgränd 41 (tidigare Stånghol...,"SSA/2060: S:t Örjans skolor 1971-1985, 1991-2009.","S:t Örjans ro 1971-1985, \r\nÖsterholms ro 198...",Österholmsskolan (ro) 1985-1990,Specialskola: Grundskola åk 1-6\r\n\r\nÖppnad ...,SSA/2060,[\r\nSankt Örjans skolor\r\n ],yvFak6FyxD7jGnHCzFdDwK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,Visa/kunskapsskolan-tyreso/e2dbbd7c-a06a-47ce-...,Kunskapsskolan Tyresö,,NaN,NaN,NaN,Solkraftsvägen 15,SSA/7093: e-arkiv: slutbetyg 2012-2021.,NaN,I övrigt på skolan.,Fristående grundskola åk 6-9.\r\n\r\nSkolan st...,SSA/7093,[\r\nKunskapsskolan Tyresö\r\n ],QT9IqGdcYKw30Ej4NiZHv2
1350,Visa/sodermalms-privata-skola/0f0c2328-a79d-46...,Södermalms privata skola,,NaN,NaN,NaN,1938 anges Folkungagatan 55,Folkskoledirektionen (SSA/0569) - E III a3 : d...,NaN,NaN,Privatskola,SSA/0569,[\r\nStockholms folkskoledirektion\r\n ],nbLG5ZXzDAN2NpJx6FBIs5
1351,Visa/olof-fryxells-flickskola/7c1a21e8-8f27-45...,Olof Fryxells flickskola,Bataljonspredikanten Olof Fryxells flickskola,NaN,NaN,NaN,Järntorget 80,NaN,NaN,okänt,Anges finnas 1848 på given adress i Staden mel...,NaN,NaN,NaN
1352,Visa/martinskolan/fcb2590d-bda7-4c70-9616-fee8...,Martinskolan,Martinskolan Söders waldorfskola 7 / (Martinsk...,NaN,NaN,NaN,Munstycksvägen 18,SSA/7059: e-arkiv: slutbetyg 2012-2021. Lucka ...,NaN,I övrigt på skolan.,"Fristående Waldorfskola åk F-9.\r\nHuvudman, S...",SSA/7059,[\r\nMartinskolan\r\n ],nek8SGk6lKoEANtUXfRJl3


In [27]:
dfTot.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1354 entries, 0 to 1353
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Url                    1354 non-null   object
 1   Namn                   1354 non-null   object
 2   Alt. namn              1354 non-null   object
 3   Byggår och arkitekt    325 non-null    object
 4   Alternativt namn       0 non-null      object
 5   Annexskolor            204 non-null    object
 6   Adress                 1354 non-null   object
 7   Arkiv på Stadsarkivet  947 non-null    object
 8   Rektorsområde          262 non-null    object
 9   Arkiv hos övriga       731 non-null    object
 10  Historik               1354 non-null   object
 11  SSA                    898 non-null    object
 12  RAarkivnamn            796 non-null    object
 13  agentid                796 non-null    object
dtypes: object(14)
memory usage: 148.2+ KB


In [24]:
dfTot.to_csv("SkolregistretStockholmsStadsarkiv_allt2.csv")

In [25]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-05-14 16:45:13.087574
Time elapsed (hh:mm:ss.ms) 0:22:58.057562


### Riksarkivet har nytt API
* GITHUB [Riksarkivet/dataplattform/wiki/OAI-PMH](https://github.com/Riksarkivet/dataplattform/wiki/OAI-PMH)
* exempel https://oai-pmh.riksarkivet.se/OAI?verb=GetRecord&identifier=SE/SSA/7131&metadataPrefix=oai_ape_ead

verkar dock vara XML ...

* skickade in en fråga om dom har någon struktur så man kan hämta ut alla arkiv om skolor se [Riksarkivet/dataplattform/issues/1](https://github.com/Riksarkivet/dataplattform/issues/1)
  * diskussion i WD hur vi hanterar folksskolor, realskolor etc [länk](https://www.wikidata.org/wiki/Talk:Q89788826)


In [29]:
testurl = "https://oai-pmh.riksarkivet.se/OAI?verb=GetRecord&identifier=SE/SSA/7131&metadataPrefix=oai_ape_ead"

In [31]:
import xml.etree.ElementTree as ET

In [42]:
import xmltodict
resRAxml = req.request('GET', testurl)
xmldict = xmltodict.parse(resRAxml.data)
#print(xmldict)
#print(xmldict) # archdesc did unittitle
print(xmldict.keys())

odict_keys(['OAI-PMH'])


In [45]:
print(xmldict['OAI-PMH'].keys())

odict_keys(['@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', 'responseDate', 'request', 'GetRecord'])


In [48]:
print(xmldict['OAI-PMH']['GetRecord'].keys())

odict_keys(['record'])


In [51]:
print(xmldict['OAI-PMH']['GetRecord']['record'].keys())

odict_keys(['header', 'metadata'])


In [53]:
print(xmldict['OAI-PMH']['GetRecord']['record']['metadata'].keys())

odict_keys(['ead'])


In [54]:
print(xmldict['OAI-PMH']['GetRecord']['record']['metadata']['ead'].keys())

odict_keys(['@xmlns:xlink', '@xmlns:xsi', '@xmlns', '@xsi:schemaLocation', 'eadheader', 'archdesc'])


In [55]:
print(xmldict['OAI-PMH']['GetRecord']['record']['metadata']['ead']['archdesc'].keys())

odict_keys(['@level', '@type', '@encodinganalog', '@relatedencoding', 'did', 'scopecontent', 'accessrestrict', 'userestrict', 'otherfindaid', 'dsc'])


In [56]:
print(xmldict['OAI-PMH']['GetRecord']['record']['metadata']['ead']['archdesc']['did'].keys())

odict_keys(['unitid', 'unittitle'])


In [58]:
print(xmldict['OAI-PMH']['GetRecord']['record']['metadata']['ead']['archdesc']['did']['unittitle'])

Carlgrenska Waldorfgymnasiet


In [64]:
print(xmldict['OAI-PMH']['GetRecord']['record']['metadata']['ead']['archdesc']['scopecontent'])

OrderedDict([('@encodinganalog', 'summary'), ('p', 'Carlgrenska Waldorfgymnasiet är en friskola i Stockholm. Friskolans slutbetyg från och med år 2016 finns på Stockholms stadsarkiv. Delar av arkivet har även överlämnats på papper. Studiebevis som saknas i e-arkiv kan finnas på papper. Dessutom finns betygskataloger på papper.')])


In [ ]:
# Tips finns en xmlviewer https://codebeautify.org/xmlviewer